In [ ]:
#---------------------------------------------------------------Import_Libraries---------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , KFold
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

In [ ]:
#------------------------------------------------------------------Read_Data-------------------------------------------------------------------
path = '/content/iris.data' # path of the dataset
Data = pd.read_csv(path, header = None) # Read CSV Data File

In [ ]:
#--------------------------------------------------------------Initial_Parameters--------------------------------------------------------------
# Define Initial Parameters
Repeats = 2
k = 4

Accuracy_on_ML_Test  = 0
Accuracy_on_ML_Train = 0
Accuracy_on_Test_Kfold = 0
Accuracy_on_Train_Kfold = 0
Accuracy_on_Test_Diagonal = 0
Accuracy_on_Train_Diagonal = 0
Accuracy_on_Test_Kfold_Diagonal = 0
Accuracy_on_Train_Kfold_Diagonal = 0
Accuracy_on_Test_Naive = 0
Accuracy_on_Train_Naive = 0
Accuracy_on_Test_Naive_Kfold = 0
Accuracy_on_Train_Naive_Kfold = 0

Conf_Mat_on_ML_Test = 0
Conf_Mat_on_ML_Train = 0
Conf_Mat_on_Test_Diagonal = 0
Conf_Mat_on_Train_Diagonal = 0
Conf_Mat_on_Test_Naive = 0
Conf_Mat_on_Train_Naive = 0
Conf_Mat_on_Test_Kfold = 0
Conf_Mat_on_Train_Kfold = 0
Conf_Mat_on_Test_Kfold_Diagonal = 0
Conf_Mat_on_Train_Kfold_Diagonal = 0
Conf_Mat_on_Test_Naive_Kfold = 0
Conf_Mat_on_Train_Naive_Kfold = 0

Group_Train_Class1 = []
Group_Test_Class1  = []
Group_Train_Class2 = []
Group_Test_Class2  = []
Group_Train_Class3 = []
Group_Test_Class3  = []
temp_mat = []
count = 0

In [ ]:
#-------------------------------------------------------------------Main_Program--------------------------------------------------------------
for j in range(Repeats) : # Repeat the program 2 times
    count += 1

    #------------------------------------------------------------Maximum_Likelihood-----------------------------------------------------------
    # Train Test Split
    [X1_Train , X1_Test , Y1_Train , Y1_Test] = train_test_split( Data.iloc[0:50,0:4]    , Data.iloc[0:50,4]    , test_size = 0.3) # Split Data into class 1
    [X2_Train , X2_Test , Y2_Train , Y2_Test] = train_test_split( Data.iloc[50:100,0:4]  , Data.iloc[50:100,4]  , test_size = 0.3) # Split Data into class 2
    [X3_Train , X3_Test , Y3_Train , Y3_Test] = train_test_split( Data.iloc[100:150,0:4] , Data.iloc[100:150,4] , test_size = 0.3) # Split Data into class 3

    X_Train = pd.concat([X1_Train , X2_Train , X3_Train]) # Merge Input Training Data
    X_Test  = pd.concat([X1_Test  , X2_Test  , X3_Test]) # Merge Input Test Data
    Y_Train = pd.concat([Y1_Train , Y2_Train , Y3_Train]) # Merge Output Training Data
    Y_Test  = pd.concat([Y1_Test  , Y2_Test  , Y3_Test]) # Merge Output Test Data

    # Find Mean & Variance
    Mean_Class1  = np.mean(X1_Train) # Find the Mean of each class
    Mean_Class2  = np.mean(X2_Train)
    Mean_Class3  = np.mean(X3_Train)

    Covar_Class1 = np.cov(X1_Train.T) # Find the Covariance of each class
    Covar_Class2 = np.cov(X2_Train.T)
    Covar_Class3 = np.cov(X3_Train.T)

    # Maximum Likelihood on Test & Train
    Probability_Class_1_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class &
    Probability_Class_2_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class2 , cov = Covar_Class2 ) # & estimate the outputs for Test Data
    Probability_Class_3_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class3 , cov = Covar_Class3 )

    Probability_Class_1_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class &
    Probability_Class_2_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class2 , cov = Covar_Class2 ) # & estimate the outputs for Training Data
    Probability_Class_3_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class3 , cov = Covar_Class3 )

    # Confusion Matrix & Accuracy ML on Test
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]) :
      temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
      max_value = max(temp)
      if temp.index(max_value) == 0 :
        Test_Predict.append('Iris-setosa')
      elif temp.index(max_value) == 1 :
        Test_Predict.append('Iris-versicolor')
      else :
        Test_Predict.append('Iris-virginica')  
      if Test_Predict[i] == Y_Test.iloc[i]: # Calculate the correct & wrong estimations
        Correct += 1
      else :
        Wrong += 1

    Conf_Mat_on_ML_Test += 100 * confusion_matrix(Y_Test.values , Test_Predict)/X_Test.shape[0]/Repeats # Confussion Matrix
    Accuracy_on_ML_Test += (100 * Correct/(Correct + Wrong))/Repeats # Accuracy
      
    # Accuracy ML on Train
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]) :
      temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]]  # Find the best class(max prob) of each Sample
      max_value = max(temp)
      if (temp.index(max_value) + 1) == 1:
       Train_Predict.append('Iris-setosa')
      elif (temp.index(max_value) + 1) == 2:
       Train_Predict.append('Iris-versicolor')
      else :
       Train_Predict.append('Iris-virginica')
      if Train_Predict[i] == Y_Train.iloc[i]: # Calculate the correct & wrong estimations
       Correct += 1
      else:
       Wrong += 1

    Conf_Mat_on_ML_Train += 100 * confusion_matrix(Y_Train.values , Train_Predict)/X_Train.shape[0]/Repeats # Confussion Matrix
    Accuracy_on_ML_Train += (100 * Correct/(Correct + Wrong))/Repeats # Accuracy

    #----------------------------------------------------Diagonal_Maximum_Likelihood----------------------------------------------------------

    # Diagonal Variance
    Covar_Class1 = np.diag(np.diag(Covar_Class1)) # Find the Diagonal Covariance of each class
    Covar_Class2 = np.diag(np.diag(Covar_Class2))
    Covar_Class3 = np.diag(np.diag(Covar_Class3))

    # Diagonal Maximum Likelihood on Test & Train
    Probability_Class_1_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class &
    Probability_Class_2_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class2 , cov = Covar_Class2 ) # & estimate the outputs for Test Data
    Probability_Class_3_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class3 , cov = Covar_Class3 )

    Probability_Class_1_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class &
    Probability_Class_2_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class2 , cov = Covar_Class2 ) # & estimate the outputs for Training Data
    Probability_Class_3_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class3 , cov = Covar_Class3 )

    # Accuracy on Diagonal Diagonal Maximum Likelihood on Test
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]):
      temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
      max_value = max(temp)
      if temp.index(max_value) == 0 :
       Test_Predict.append('Iris-setosa')
      elif temp.index(max_value) == 1 :
       Test_Predict.append('Iris-versicolor')
      else :
       Test_Predict.append('Iris-virginica')
      if Test_Predict[i] == Y_Test.iloc[i] :
        Correct += 1
      else :
        Wrong += 1

    Conf_Mat_on_Test_Diagonal += 100 * confusion_matrix(Y_Test.values , Test_Predict)/X_Test.shape[0]/Repeats
    Accuracy_on_Test_Diagonal += (100 * Correct/(Correct + Wrong))/Repeats
       
    # Accuracy on Diagonal Diagonal Maximum Likelihood on Train
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]):
      temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]] # Find the best class(max prob) of each Sample
      max_value = max(temp)
      if (temp.index(max_value) + 1) == 1 :
       Train_Predict.append('Iris-setosa')
      elif (temp.index(max_value) + 1) == 2 :
       Train_Predict.append('Iris-versicolor')
      else :
       Train_Predict.append('Iris-virginica')
      if Train_Predict[i] == Y_Train.iloc[i] :
       Correct += 1
      else :
       Wrong += 1

    Conf_Mat_on_Train_Diagonal += 100 * confusion_matrix(Y_Train.values , Train_Predict)/X_Train.shape[0]/Repeats
    Accuracy_on_Train_Diagonal += (100 * Correct/(Correct + Wrong))/Repeats

    #----------------------------------------------------Naive_Bayes--------------------------------------------------------------------------

    # Find Mean & Variance
    Mean_Class1_X1  = np.mean(X1_Train.iloc[:,0]) # Find the Mean of each class & each Feature
    Mean_Class1_X2  = np.mean(X1_Train.iloc[:,1])
    Mean_Class1_X3  = np.mean(X1_Train.iloc[:,2])
    Mean_Class1_X4  = np.mean(X1_Train.iloc[:,3])
    Mean_Class2_X1  = np.mean(X2_Train.iloc[:,0])
    Mean_Class2_X2  = np.mean(X2_Train.iloc[:,1])
    Mean_Class2_X3  = np.mean(X2_Train.iloc[:,2])
    Mean_Class2_X4  = np.mean(X2_Train.iloc[:,3])
    Mean_Class3_X1  = np.mean(X3_Train.iloc[:,0])
    Mean_Class3_X2  = np.mean(X3_Train.iloc[:,1])
    Mean_Class3_X3  = np.mean(X3_Train.iloc[:,2])
    Mean_Class3_X4  = np.mean(X3_Train.iloc[:,3])

    Covar_Class1_X1 = np.cov((X1_Train.iloc[:,0]).T) # Find the Covariance of each class & each Feature
    Covar_Class1_X2 = np.cov((X1_Train.iloc[:,1]).T)
    Covar_Class1_X3 = np.cov((X1_Train.iloc[:,2]).T)
    Covar_Class1_X4 = np.cov((X1_Train.iloc[:,3]).T)
    Covar_Class2_X1 = np.cov((X2_Train.iloc[:,0]).T)
    Covar_Class2_X2 = np.cov((X2_Train.iloc[:,1]).T)
    Covar_Class2_X3 = np.cov((X2_Train.iloc[:,2]).T)
    Covar_Class2_X4 = np.cov((X2_Train.iloc[:,3]).T)
    Covar_Class3_X1 = np.cov((X3_Train.iloc[:,0]).T)
    Covar_Class3_X2 = np.cov((X3_Train.iloc[:,1]).T)
    Covar_Class3_X3 = np.cov((X3_Train.iloc[:,2]).T)
    Covar_Class3_X4 = np.cov((X3_Train.iloc[:,3]).T)

    # Naive_Bayes on Test
    Probability_Class_1_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class1_X1 , cov = Covar_Class1_X1 ) # Make a Gaussian PDF for each class &
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class1_X2 , cov = Covar_Class1_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class1_X3 , cov = Covar_Class1_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class1_X4 , cov = Covar_Class1_X4 ))
    
    Probability_Class_2_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class2_X1 , cov = Covar_Class2_X1 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class2_X2 , cov = Covar_Class2_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class2_X3 , cov = Covar_Class2_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class2_X4 , cov = Covar_Class2_X4 ))

    Probability_Class_3_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class3_X1 , cov = Covar_Class3_X1 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class3_X2 , cov = Covar_Class3_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class3_X3 , cov = Covar_Class3_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class3_X4 , cov = Covar_Class3_X4 ))
    
    Probability_Class_1_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class1_X1 , cov = Covar_Class1_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class1_X2 , cov = Covar_Class1_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class1_X3 , cov = Covar_Class1_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class1_X4 , cov = Covar_Class1_X4 ))

    Probability_Class_2_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class2_X1 , cov = Covar_Class2_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class2_X2 , cov = Covar_Class2_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class2_X3 , cov = Covar_Class2_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class2_X4 , cov = Covar_Class2_X4 ))

    Probability_Class_3_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class3_X1 , cov = Covar_Class3_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class3_X2 , cov = Covar_Class3_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class3_X3 , cov = Covar_Class3_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class3_X4 , cov = Covar_Class3_X4 ))


    # Accuracy ML on Test Naive
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]):
        temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if temp.index(max_value) == 0 :
            Test_Predict.append('Iris-setosa')
        elif temp.index(max_value) == 1 :
            Test_Predict.append('Iris-versicolor')
        else :
            Test_Predict.append('Iris-virginica')
        if Test_Predict[i] == Y_Test.iloc[i] :
            Correct += 1
        else :
            Wrong += 1

    Conf_Mat_on_Test_Naive += 100 * confusion_matrix(Y_Test.values , Test_Predict)/X_Test.shape[0]/Repeats
    Accuracy_on_Test_Naive += (100 * Correct/(Correct + Wrong))/Repeats

    # Accuracy ML on Train Naive
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]):
        temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if temp.index(max_value) == 0 :
            Train_Predict.append('Iris-setosa')
        elif temp.index(max_value) == 1 :
            Train_Predict.append('Iris-versicolor')
        else :
            Train_Predict.append('Iris-virginica')
        if Train_Predict[i] == Y_Train.iloc[i] :
            Correct += 1
        else :
            Wrong += 1

    Conf_Mat_on_Train_Naive += 100 * confusion_matrix(Y_Train.values , Train_Predict)/X_Train.shape[0]/Repeats
    Accuracy_on_Train_Naive += (100 * Correct/(Correct + Wrong))/Repeats


In [ ]:
#-----------------------------------------------------------------KFold_Main_Program---------------------------------------------------------
for i in range(50) :
    temp_mat.append(i)

kf = KFold(n_splits=k , shuffle = True , random_state = 1)
count = 0 
for train_index_class1 , test_index_class1 in kf.split(temp_mat) :
    count += 1 
    Group_Train_Class1.append(train_index_class1) 
    Group_Test_Class1.append(test_index_class1) 

kf = KFold(n_splits=k , shuffle = True , random_state = 2)
count = 0 
for train_index_class2 , test_index_class2 in kf.split(temp_mat) :
    count += 1 
    Group_Train_Class2.append(train_index_class2) 
    Group_Test_Class2.append(test_index_class2) 

kf = KFold(n_splits=k , shuffle = True , random_state = 3)
count = 0 
for train_index_class3 , test_index_class3 in kf.split(temp_mat) :
    count += 1 
    Group_Train_Class3.append(train_index_class3) 
    Group_Test_Class3.append(test_index_class3) 


for j in range(4) :
    #----------------------------------------------------------KFold_Maximum_Likelihood---------------------------------------------------------
    # Train Test Split
    X_Train = pd.concat([Data.iloc[Group_Train_Class1[j],0:4] , Data.iloc[Group_Train_Class2[j]+50,0:4] , Data.iloc[Group_Train_Class3[j]+100,0:4]])
    X_Test  = pd.concat([Data.iloc[Group_Test_Class1[j],0:4]  , Data.iloc[Group_Test_Class2[j]+50,0:4]  , Data.iloc[Group_Test_Class3[j]+100,0:4]])
    Y_Train = pd.concat([Data.iloc[Group_Train_Class1[j],4]   , Data.iloc[Group_Train_Class2[j]+50,4]   , Data.iloc[Group_Train_Class3[j]+100,4]])
    Y_Test  = pd.concat([Data.iloc[Group_Test_Class1[j],4]    , Data.iloc[Group_Test_Class2[j]+50,4]    , Data.iloc[Group_Test_Class3[j]+100,4]])

    # Find Mean & Variance
    Mean_Class1  = np.mean(Data.iloc[Group_Train_Class1[j],0:4])
    Mean_Class2  = np.mean(Data.iloc[Group_Train_Class2[j]+50,0:4])
    Mean_Class3  = np.mean(Data.iloc[Group_Train_Class3[j]+100,0:4])

    Covar_Class1 = np.cov((Data.iloc[Group_Train_Class1[j],0:4]).T)
    Covar_Class2 = np.cov((Data.iloc[Group_Train_Class2[j]+50,0:4]).T)
    Covar_Class3 = np.cov((Data.iloc[Group_Train_Class3[j]+100,0:4]).T)

    # ML on Test
    Probability_Class_1_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class &
    Probability_Class_2_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class2 , cov = Covar_Class2 )
    Probability_Class_3_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class3 , cov = Covar_Class3 )

    Probability_Class_1_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class1 , cov = Covar_Class1 )
    Probability_Class_2_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class2 , cov = Covar_Class2 )
    Probability_Class_3_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class3 , cov = Covar_Class3 )

    # Accuracy ML on Test
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]):
        temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if (temp.index(max_value) + 1) == 1 :
            Test_Predict.append('Iris-setosa')
        elif (temp.index(max_value) + 1) == 2 :
            Test_Predict.append('Iris-versicolor')
        else :
            Test_Predict.append('Iris-virginica')
        if Test_Predict[i] == Y_Test.iloc[i] :
            Correct += 1
        else :
            Wrong += 1
    
    Conf_Mat_on_Test_Kfold += 100 * confusion_matrix(Y_Test.values , Test_Predict)/(X_Test.shape[0])/k    
    Accuracy_on_Test_Kfold += (100 * Correct/(Correct + Wrong))/k

    # Accuracy ML on Train
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]):
        temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if (temp.index(max_value) + 1) == 1 :
            Train_Predict.append('Iris-setosa')
        elif (temp.index(max_value) + 1) == 2 :
            Train_Predict.append('Iris-versicolor')
        else :
            Train_Predict.append('Iris-virginica')
        if Train_Predict[i] == Y_Train.iloc[i] :
            Correct += 1
        else :
            Wrong += 1

    Conf_Mat_on_Train_Kfold += 100 * confusion_matrix(Y_Train.values , Train_Predict)/(X_Train.shape[0])/k
    Accuracy_on_Train_Kfold += (100 * Correct/(Correct + Wrong))/k

    #----------------------------------------------------------KFold_Diagonal_Maximum_Likelihood----------------------------------------------------------------------

    Covar_Class1 = np.diag(np.diag(Covar_Class1))
    Covar_Class2 = np.diag(np.diag(Covar_Class2))
    Covar_Class3 = np.diag(np.diag(Covar_Class3))

    # ML on Test
    Probability_Class_1_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class1 , cov = Covar_Class1 ) # Make a Gaussian PDF for each class
    Probability_Class_2_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class2 , cov = Covar_Class2 )
    Probability_Class_3_Test = multivariate_normal.pdf( X_Test , mean = Mean_Class3 , cov = Covar_Class3 )

    Probability_Class_1_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class1 , cov = Covar_Class1 )
    Probability_Class_2_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class2 , cov = Covar_Class2 )
    Probability_Class_3_Train = multivariate_normal.pdf( X_Train , mean = Mean_Class3 , cov = Covar_Class3 )

    # Accuracy ML on Test
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]):
        temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if (temp.index(max_value) + 1) == 1 :
            Test_Predict.append('Iris-setosa')
        elif (temp.index(max_value) + 1) == 2 :
            Test_Predict.append('Iris-versicolor')
        else :
            Test_Predict.append('Iris-virginica')
        if Test_Predict[i] == Y_Test.iloc[i] :
            Correct += 1
        else :
            Wrong += 1

    Conf_Mat_on_Test_Kfold_Diagonal += 100 * confusion_matrix(Y_Test.values , Test_Predict)/(X_Test.shape[0])/k
    Accuracy_on_Test_Kfold_Diagonal += (100 * Correct/(Correct + Wrong))/k

    # Accuracy ML on Train
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]):
        temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if temp.index(max_value) == 0:
            Train_Predict.append('Iris-setosa')
        elif temp.index(max_value) == 1:
            Train_Predict.append('Iris-versicolor')
        else :
            Train_Predict.append('Iris-virginica')
        if Train_Predict[i] == Y_Train.iloc[i]:
            Correct += 1
        else :
            Wrong += 1
   
    Conf_Mat_on_Train_Kfold_Diagonal += 100 * confusion_matrix(Y_Train.values , Train_Predict)/(X_Train.shape[0])/k  
    Accuracy_on_Train_Kfold_Diagonal += (100 * Correct/(Correct + Wrong))/k

    #-----------------------------------------------------------KFold_Naive_Bayes---------------------------------------------------------------------    

    # Find Mean & Variance
    Mean_Class1_X1  = np.mean(Data.iloc[Group_Train_Class1[j],0])
    Mean_Class1_X2  = np.mean(Data.iloc[Group_Train_Class1[j],1])
    Mean_Class1_X3  = np.mean(Data.iloc[Group_Train_Class1[j],2])
    Mean_Class1_X4  = np.mean(Data.iloc[Group_Train_Class1[j],3])
    Mean_Class2_X1  = np.mean(Data.iloc[Group_Train_Class2[j]+50,0])
    Mean_Class2_X2  = np.mean(Data.iloc[Group_Train_Class2[j]+50,1])
    Mean_Class2_X3  = np.mean(Data.iloc[Group_Train_Class2[j]+50,2])
    Mean_Class2_X4  = np.mean(Data.iloc[Group_Train_Class2[j]+50,3])
    Mean_Class3_X1  = np.mean(Data.iloc[Group_Train_Class3[j]+100,0])
    Mean_Class3_X2  = np.mean(Data.iloc[Group_Train_Class3[j]+100,1])
    Mean_Class3_X3  = np.mean(Data.iloc[Group_Train_Class3[j]+100,2])
    Mean_Class3_X4  = np.mean(Data.iloc[Group_Train_Class3[j]+100,3])
    Covar_Class1_X1 = np.cov((Data.iloc[Group_Train_Class1[j],0]).T)
    Covar_Class1_X2 = np.cov((Data.iloc[Group_Train_Class1[j],1]).T)
    Covar_Class1_X3 = np.cov((Data.iloc[Group_Train_Class1[j],2]).T)
    Covar_Class1_X4 = np.cov((Data.iloc[Group_Train_Class1[j],3]).T)
    Covar_Class2_X1 = np.cov((Data.iloc[Group_Train_Class2[j]+50,0]).T)
    Covar_Class2_X2 = np.cov((Data.iloc[Group_Train_Class2[j]+50,1]).T)
    Covar_Class2_X3 = np.cov((Data.iloc[Group_Train_Class2[j]+50,2]).T)
    Covar_Class2_X4 = np.cov((Data.iloc[Group_Train_Class2[j]+50,3]).T)
    Covar_Class3_X1 = np.cov((Data.iloc[Group_Train_Class3[j]+100,0]).T)
    Covar_Class3_X2 = np.cov((Data.iloc[Group_Train_Class3[j]+100,1]).T)
    Covar_Class3_X3 = np.cov((Data.iloc[Group_Train_Class3[j]+100,2]).T)
    Covar_Class3_X4 = np.cov((Data.iloc[Group_Train_Class3[j]+100,3]).T)

    # ML on Test Naive
    Probability_Class_1_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class1_X1 , cov = Covar_Class1_X1 ) # Make a Gaussian PDF for each class &
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class1_X2 , cov = Covar_Class1_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class1_X3 , cov = Covar_Class1_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class1_X4 , cov = Covar_Class1_X4 ))

    Probability_Class_2_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class2_X1 , cov = Covar_Class2_X1 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class2_X2 , cov = Covar_Class2_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class2_X3 , cov = Covar_Class2_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class2_X4 , cov = Covar_Class2_X4 ))

    Probability_Class_3_Test = (multivariate_normal.pdf( X_Test.iloc[:,0] , mean = Mean_Class3_X1 , cov = Covar_Class3_X1 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,1] , mean = Mean_Class3_X2 , cov = Covar_Class3_X2 ) 
                              * multivariate_normal.pdf( X_Test.iloc[:,2] , mean = Mean_Class3_X3 , cov = Covar_Class3_X3 )
                              * multivariate_normal.pdf( X_Test.iloc[:,3] , mean = Mean_Class3_X4 , cov = Covar_Class3_X4 ))

    Probability_Class_1_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class1_X1 , cov = Covar_Class1_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class1_X2 , cov = Covar_Class1_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class1_X3 , cov = Covar_Class1_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class1_X4 , cov = Covar_Class1_X4 ))

    Probability_Class_2_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class2_X1 , cov = Covar_Class2_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class2_X2 , cov = Covar_Class2_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class2_X3 , cov = Covar_Class2_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class2_X4 , cov = Covar_Class2_X4 ))

    Probability_Class_3_Train = (multivariate_normal.pdf( X_Train.iloc[:,0] , mean = Mean_Class3_X1 , cov = Covar_Class3_X1 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,1] , mean = Mean_Class3_X2 , cov = Covar_Class3_X2 ) 
                               * multivariate_normal.pdf( X_Train.iloc[:,2] , mean = Mean_Class3_X3 , cov = Covar_Class3_X3 )
                               * multivariate_normal.pdf( X_Train.iloc[:,3] , mean = Mean_Class3_X4 , cov = Covar_Class3_X4 ))
    
    # Accuracy ML on Test Naive
    Test_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Test.shape[0]):
        temp = [Probability_Class_1_Test[i] , Probability_Class_2_Test[i] , Probability_Class_3_Test[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if temp.index(max_value) == 0 :
            Test_Predict.append('Iris-setosa')
        elif temp.index(max_value) == 1 :
            Test_Predict.append('Iris-versicolor')
        else :
            Test_Predict.append('Iris-virginica')
        if Test_Predict[i] == Y_Test.iloc[i] :
            Correct += 1
        else :
            Wrong += 1
    
    Conf_Mat_on_Test_Naive_Kfold += 100 * confusion_matrix(Y_Test.values , Test_Predict)/X_Test.shape[0]/k
    Accuracy_on_Test_Naive_Kfold += (100 * Correct/(Correct + Wrong))/k

    # Accuracy ML on Train Naive
    Train_Predict = []
    Correct = 0
    Wrong   = 0
    for i in range(X_Train.shape[0]):
        temp = [Probability_Class_1_Train[i] , Probability_Class_2_Train[i] , Probability_Class_3_Train[i]] # Find the best class(max prob) of each Sample
        max_value = max(temp)
        if (temp.index(max_value) + 1) == 1 :
            Train_Predict.append('Iris-setosa')
        elif (temp.index(max_value) + 1) == 2 :
            Train_Predict.append('Iris-versicolor')
        else :
            Train_Predict.append('Iris-virginica')
        if Train_Predict[i] == Y_Train.iloc[i] :
            Correct += 1
        else :
            Wrong += 1
   
    Conf_Mat_on_Train_Naive_Kfold += 100 * confusion_matrix(Y_Train.values , Train_Predict)/X_Train.shape[0]/k
    Accuracy_on_Train_Naive_Kfold += (100 * Correct/(Correct + Wrong))/k


In [ ]:
#-------------------------------------------------------Print_Accuracies_&_Confusion Matrixes-------------------------------------------------------

print("Maximum Likelihood Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_ML_Test,2) )
print("Maximum Likelihood Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_ML_Test.round(2) )
print("Maximum Likelihood Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_ML_Train,2) )
print("Maximum Likelihood Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_ML_Train.round(2) )

print("K-Fold Maximum Likelihood Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_Test_Kfold,2) )
print("K-Fold Maximum Likelihood Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Test_Kfold.round(2) )
print("K-Fold Maximum Likelihood Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_Train_Kfold,2) )
print("K-Fold Maximum Likelihood Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Train_Kfold.round(2) )

print("Diagonal Maximum Likelihood Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_Test_Diagonal,2) )
print("Diagonal Maximum Likelihood Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Test_Diagonal.round(2) )
print("Diagonal Maximum Likelihood Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_Train_Diagonal,2) )
print("Diagonal Maximum Likelihood Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Train_Diagonal.round(2) )

print("Diagonal K-Fold Maximum Likelihood Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_Test_Kfold_Diagonal,2) )
print("Diagonal K-Fold Maximum Likelihood Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Test_Kfold_Diagonal.round(2) )
print("Diagonal K-Fold Maximum Likelihood Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_Train_Kfold_Diagonal,2) )
print("Diagonal K-Fold Maximum Likelihood Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Train_Kfold_Diagonal.round(2) )

print("Naive Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_Test_Naive,2) )
print("Naive Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Test_Naive.round(2) )
print("Naive Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_Train_Naive,2) )
print("Naive Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Train_Naive.round(2) )

print("Naive K-Fold Accuracy on Test Data is (percentile) : %s \n" % round(Accuracy_on_Test_Naive_Kfold,2) )
print("Naive K-Fold Confusion Matrix on Test Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Test_Naive_Kfold.round(2) )
print("Naive K-Fold Accuracy on Train Data is (percentile) : %s \n" % round(Accuracy_on_Train_Naive_Kfold,2) )
print("Naive K-Fold Confusion Matrix on Train Data is (percentile) : \n\n %s \n\n\n\n" % Conf_Mat_on_Train_Naive_Kfold.round(2) )


Maximum Likelihood Accuracy on Test Data is (percentile) : 98.89 

Maximum Likelihood Confusion Matrix on Test Data is (percentile) : 

 [[33.33  0.    0.  ]
 [ 0.   32.22  1.11]
 [ 0.    0.   33.33]] 




Maximum Likelihood Accuracy on Train Data is (percentile) : 97.62 

Maximum Likelihood Confusion Matrix on Train Data is (percentile) : 

 [[33.33  0.    0.  ]
 [ 0.   31.9   1.43]
 [ 0.    0.95 32.38]] 




K-Fold Maximum Likelihood Accuracy on Test Data is (percentile) : 97.97 

K-Fold Maximum Likelihood Confusion Matrix on Test Data is (percentile) : 

 [[33.33  0.    0.  ]
 [ 0.   32.    1.34]
 [ 0.    0.69 32.64]] 




K-Fold Maximum Likelihood Accuracy on Train Data is (percentile) : 98.22 

K-Fold Maximum Likelihood Confusion Matrix on Train Data is (percentile) : 

 [[33.33  0.    0.  ]
 [ 0.   32.    1.33]
 [ 0.    0.44 32.89]] 




Diagonal Maximum Likelihood Accuracy on Test Data is (percentile) : 97.78 

Diagonal Maximum Likelihood Confusion Matrix on Test Data is (percen